## 利用距离分割背景 == 生成分割后的图像

In [ ]:
# 文件导入
import numpy as np 
import cv2 
import os 
import matplotlib.pyplot as plt 
import time 
import os.path as osp 

In [ ]:
#===获得文件最大序号值===#
def Get_IMG_SeqNum(root = './', img_suffix = '.npy', img_name_start = 'color', img_name_startnum = 5): 
    # root = './'  #指定目录 
    Seq_num_Png = [] 
    for files in os.listdir(root): 
        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start):  
            Seq_num_Png.append(int(os.path.splitext(files)[0][-6:])) 
    if len(Seq_num_Png)==0:  #列表为空 返回0  
        Seq_num_Png.append(0) 
    return max(Seq_num_Png) 
# 实例 = 获取最大文件序号 
Get_IMG_SeqNum(root ="./Image_NPY2/NPY_color_depth/")  

In [ ]:
#===独立数据 = 处理环节===# 
# 分割距离设置   
depth_scale = 0.0010000000474974513  
clipping_distance_in_meters = 1       #1 meter  
clipping_distance = clipping_distance_in_meters / depth_scale  


Figure_NUM = Get_IMG_SeqNum(root ="./Image_NPY/NPY_color_depth/")  # 获取处理图片数量

# Figure_NUM = 10

for Seq_num in range(Figure_NUM + 1):   # 单个文件[1522, 1511] 
    Plantname = '_HLG' 
    str_Fig_CNT_Seq = Plantname + '_' + str(Seq_num).zfill(6)      # 含植物名称

    Image_path2 = ["./Image_NPY2/NPY_color_depth/", "./Image_NPY2/NPY_Infrared/", 
                    "./Image_IMG2/IMG_color_depth/", "./Image_IMG2/IMG_Infrared/","./Image_IMG2/IMG_kcolor/","./Image_IMG2/IMG_Con_color/"] 

    # 检测路径是否存在，不存在跳过for循坏
    color_image_pathis = os.path.isfile( path= Image_path2[0] + 'color' + str_Fig_CNT_Seq + '.npy' )
    
    if (color_image_pathis):
        # 加载文件 
        color_image_npy = np.load(file = Image_path2[0] +'color'+ str_Fig_CNT_Seq +'.npy')   # 加载npy文件 
        deep_image_npy = np.load(file = Image_path2[0] +'depth'+ str_Fig_CNT_Seq +'.npy')    # 加载npy文件
        deep_image_npy_cMAP = cv2.applyColorMap(cv2.convertScaleAbs(deep_image_npy, alpha=0.03), cv2.COLORMAP_JET) #伪彩图

        # 背景去除 
        deep_image_npy_3d = np.dstack((deep_image_npy, deep_image_npy, deep_image_npy)) 
        gray_color = np.ones_like(color_image_npy) * 153 
        backg_remove = np.where( (deep_image_npy_3d > clipping_distance) | (deep_image_npy_3d < 0) , gray_color , color_image_npy ) 

        print('current image SeqNum:', str_Fig_CNT_Seq)

        # if False:     #不使用
        #     # 图像显示 
        #     images = np.hstack((backg_remove, deep_image_npy_cMAP))            
        #     images_plt = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)               
        #     plt.rcParams['figure.figsize'] = [16, 6]                         
        #     plt.imshow(images_plt)                                          

        # 图像保存
        # cv2.imwrite( Image_path2[4] + 'kcolor' + str_Fig_CNT_Seq +'.jpg', backg_remove) #保存图像数据

        # 对比图保存
        images_contrast = np.hstack(( color_image_npy, backg_remove)) 
        cv2.imwrite( Image_path2[5] + 'Concolor' + str_Fig_CNT_Seq +'.jpg', images_contrast) #保存图像数据

In [ ]:
#===创建的目录= 函数===#
def mkdir(path):
    import os      # 引入模块
    isExists=os.path.exists(path) # 判断路径是否存在 #存在True  #不存在 False
    if not isExists: # 判断结果
        os.makedirs(path) # 创建目录操作函数
        print (path +' 创建成功')
        return True
    else:  
        print (path+' 目录已存在')# 如果目录存在则不创建，并提示目录已存在
        return False

if True:
    # 定义要创建的目录2
    Image_path2 = ["./Image_NPY2/NPY_color_depth/",  "./Image_NPY2/NPY_Infrared/", 
                "./Image_IMG2/IMG_color_depth/",  "./Image_IMG2/IMG_Infrared/", "./Image_IMG2/IMG_kcolor","./Image_IMG2/IMG_Con_color/"]   
    # 调用函数2
    for path in Image_path2:
        mkdir(path)